### Goal of this program: To take inputs (stats from tennis players) and output predicted winning percentage

### Steps:
###### 1) Scrape data from tennis database (tennisabstract.com for now)
###### 2) input data into model
###### 3) Model ouputs predicted winning percentage (and maybe predicted outcome against opponent in the future)

In [1]:
import numpy as np
import random
from random import randint
from sklearn.preprocessing import MinMaxScaler
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import *
from keras.models import Model
from keras.applications import imagenet_utils
from keras.preprocessing import image
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

import os
import json
# import nbimporter


In [2]:
# from serve_scraper import data

# import DataTransfer

In [3]:
with open("data.json", "r") as file:
    data = json.load(file)
print(data)

{'Giovanni Mpetshi Perricard': ['No data?', 64.9, 15.53, 14.84, {'singles': [14.68, 77.0, -79.3, 66.22, 10], 'doubles': [14.06, -6.0, -37.23, 36.84, 3]}], 'Alexander Zverev': [126.79, 71.0, 16.15, 14.86, {'singles': [15.43, 298.0, 169.13, 76.74, 12], 'doubles': [14.29, 9.0, -12.24, 50.0, 4]}], 'Hubert Hurkacz': [126.79, 63.4, 15.82, 14.22, {'singles': [15.37, 103.0, 44.05, 67.27, 6], 'doubles': [13.94, -4.0, -7.47, 40.0, 1]}], 'Matteo Berrettini': [130.52, 68.2, 15.71, 15.44, {'singles': [15.2, 111.0, 62.5, 72.0, 10], 'doubles': [14.49, 4.0, 2.31, 100.0, 1]}], 'Jannik Sinner': [121.19, 61.4, 16.39, 15.44, {'singles': [15.51, 431.0, 305.03, 92.5, 29], 'doubles': [14.47, 18.0, 1.84, 55.56, 2]}], 'Ben Shelton': [137.97, 68.8, 15.77, 14.64, {'singles': [15.35, 76.0, 2.01, 62.12, 5], 'doubles': [14.14, -21.0, -53.49, 43.33, 3]}], 'Taylor Fritz': [123.06, 62.4, 16.12, 15.05, {'singles': [15.41, 199.0, 86.07, 69.33, 8], 'doubles': [14.19, 34.0, 1.31, 65.0, 3]}], 'Brandon Nakashima': [118.71, 

### Setting all the training/valid sets up 

In [4]:
# Data dependent on match type (singles and doubles) will be put in corresponding lists. These corresponding lists
# will all be put together in another list. This list is then paired with another list in a dictionary. For example,
# singles training sample data will be put in their corresponding lists. These lists will be put in a singles training
# sample list, which countains all the lists with the singles training sample data. This singles training list will 
# then be paired with a doubles training sample list in a dictionary. 

In [5]:
adv_training_sets = dict()
scaled_adv_training_sets = dict()


valid_sets = dict()
scaled_valid_sets = dict()

singlesUtr_train_samples = []
doublesUtr_train_samples = []
servePercentage_train_samples = []

scaled_singlesUtr_train_samples = []
scaled_doublesUtr_train_samples = []
scaled_servePercentage_train_samples = []

basic_training_sets = []
basic_training_sets.append(singlesUtr_train_samples)
basic_training_sets.append(doublesUtr_train_samples) 
basic_training_sets.append(servePercentage_train_samples)

scaled_basic_training_sets = []
scaled_basic_training_sets.append(scaled_singlesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_doublesUtr_train_samples) 
scaled_basic_training_sets.append(scaled_servePercentage_train_samples)

singlesGameDiff_train_samples = []
singlesWeightedGameDiff_train_samples = []
singlesWinPercentage_train_samples = []
singlesOpponentUtr_train_samples = []
singlesLongestWinStreak_train_samples = []

scaled_singlesGameDiff_train_samples = []
scaled_singlesWeightedGameDiff_train_samples = []
scaled_singlesWinPercentage_train_samples = []
scaled_singlesOpponentUtr_train_samples = []
scaled_singlesLongestWinStreak_train_samples = []

adv_training_sets["singles"] = [] 
adv_training_sets["singles"].append(singlesGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWeightedGameDiff_train_samples) 
adv_training_sets["singles"].append(singlesWinPercentage_train_samples) 
adv_training_sets["singles"].append(singlesOpponentUtr_train_samples) 
adv_training_sets["singles"].append(singlesLongestWinStreak_train_samples)

scaled_adv_training_sets["singles"] = [] 
scaled_adv_training_sets["singles"].append(scaled_singlesGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesWinPercentage_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesOpponentUtr_train_samples) 
scaled_adv_training_sets["singles"].append(scaled_singlesLongestWinStreak_train_samples)


# print(scaled_adv_training_sets)

'''
for i in range(5):   # Adding scaled values to all advanced stat training sets
    # list_ = adv_training_sets[match_type][i]
    scaled_list = scaled_adv_training_sets['singles'][i]
    scaled_adv_training_sets['singles'][i].extend([0, i])
    # print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

print(scaled_adv_training_sets)

print(type(scaled_singlesGameDiff_train_samples))
'''

# print(singles_train_lists)

doublesGameDiff_train_samples = []
doublesWeightedGameDiff_train_samples = []
doublesWinPercentage_train_samples = []
doublesOpponentUtr_train_samples = []
doublesLongestWinStreak_train_samples = []

scaled_doublesGameDiff_train_samples = []
scaled_doublesWeightedGameDiff_train_samples = []
scaled_doublesWinPercentage_train_samples = []
scaled_doublesOpponentUtr_train_samples = []
scaled_doublesLongestWinStreak_train_samples = []

'''
adv_training_sets["doubles"] = [doublesGameDiff_train_samples, doublesWeightedGameDiff_train_samples, doublesWinPercentage_train_samples, doublesOpponentUtr_train_samples, doublesLongestWinStreak_train_samples]
scaled_adv_training_sets["doubles"] = [scaled_doublesGameDiff_train_samples, scaled_doublesWeightedGameDiff_train_samples, scaled_doublesWinPercentage_train_samples, scaled_doublesOpponentUtr_train_samples, scaled_doublesLongestWinStreak_train_samples]
'''

adv_training_sets["doubles"] = [] 
adv_training_sets["doubles"].append(doublesGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWeightedGameDiff_train_samples) 
adv_training_sets["doubles"].append(doublesWinPercentage_train_samples) 
adv_training_sets["doubles"].append(doublesOpponentUtr_train_samples) 
adv_training_sets["doubles"].append(doublesLongestWinStreak_train_samples)
                                    
scaled_adv_training_sets["doubles"] = [] 
scaled_adv_training_sets["doubles"].append(scaled_doublesGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWeightedGameDiff_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesWinPercentage_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesOpponentUtr_train_samples) 
scaled_adv_training_sets["doubles"].append(scaled_doublesLongestWinStreak_train_samples)

serveSpeed_train_labels = []
scaled_serveSpeed_train_labels = []

##########################################################################################################################################
'''
singlesUtr_valid_samples = []
doublesUtr_valid_samples = []
servePercentage_valid_samples = []

singlesGameDiff_valid_samples = []
singlesWeightedGameDiff_valid_samples = []
singlesWinPercentage_valid_samples = []
singlesOpponentUtr_valid_samples = []
singlesLongestWinStreak_valid_samples = []

valid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]


doublesGameDiff_valid_samples = []
doublesWeightedGameDiff_valid_samples = []
doublesWinPercentage_valid_samples = []
doublesOpponentUtr_valid_samples = []
doublesLongestWinStreak_valid_samples = []

valid_sets["doubles"] = [doublesGameDiff_valid_samples, doublesWeightedGameDiff_valid_samples, doublesWinPercentage_valid_samples, doublesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]
scaled_valid_sets["doubles"] = [scaled_doublesGameDiff_valid_samples, scaled_doublesWeightedGameDiff_valid_samples, scaled_doublesWinPercentage_valid_samples, scaled_doublesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]
'''


# serveSpeed_valid_labels = []

'\nsinglesUtr_valid_samples = []\ndoublesUtr_valid_samples = []\nservePercentage_valid_samples = []\n\nsinglesGameDiff_valid_samples = []\nsinglesWeightedGameDiff_valid_samples = []\nsinglesWinPercentage_valid_samples = []\nsinglesOpponentUtr_valid_samples = []\nsinglesLongestWinStreak_valid_samples = []\n\nvalid_sets["singles"] = [singlesGameDiff_valid_samples, singlesWeightedGameDiff_valid_samples, singlesWinPercentage_valid_samples, singlesOpponentUtr_valid_samples, singlesLongestWinStreak_valid_samples]\nscaled_valid_sets["singles"] = [scaled_singlesGameDiff_valid_samples, scaled_singlesWeightedGameDiff_valid_samples, scaled_singlesWinPercentage_valid_samples, scaled_singlesOpponentUtr_valid_samples, scaled_singlesLongestWinStreak_valid_samples]\n\n\ndoublesGameDiff_valid_samples = []\ndoublesWeightedGameDiff_valid_samples = []\ndoublesWinPercentage_valid_samples = []\ndoublesOpponentUtr_valid_samples = []\ndoublesLongestWinStreak_valid_samples = []\n\nvalid_sets["doubles"] = [doub

In [6]:
print(len(data))

76


### Getting Training and Varification sets data from data dictionary

In [7]:
# Need to do: get all data from data dict into training_labels and training_sameples
# Each stat can go in its individual list, I will just have to be careful that every individual number (Stat) for each player is the same 
# index in each list. For example, Sinner's serve percentage will have the same index as his singles UTR, doubles UTR, etc. 

In [8]:
# PS - only lists with a string ("no data?") are the serve speed lists!

name_list = list(data)

match_types = ["singles", "doubles"]

# Training Set
for i in range(70):
    
    training_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_train_labels.append(player[0])   # appending serve speed to label list

    for i in range(3):
        # print(player[i+1])
        if player[i+1] != 0:
            basic_training_sets[i].append(player[i+1])  # appending serve percentage, singles utr, and doubles utr to training set lists
        else:
            basic_training_sets[i].append(None)    # If singles or doubles UTR, or serve percentage equal 0, append None to the training set
    '''   
    servePercentage_train_samples.append(player[1])
    singlesUtr_train_samples.append(player[2])
    doublesUtr_train_samples.append(player[3])
    '''

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        print(match_type_data)
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in adv_training_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): # Append data from data dict to appropriate training set lists
                adv_training_sets[dict_string][i].append(match_type_data[i])   
        

# Validation Set ##################################################################################################################
'''
for i in range(10):
    valid_list = []
    name = random.choice(name_list)
    name_list.remove(name)
    player = data[name]
    # print(player)

    serveSpeed_valid_labels.append(player[0])

    servePercentage_valid_samples.append(player[1])
    singlesUtr_valid_samples.append(player[2])
    doublesUtr_valid_samples.append(player[3])

    for match_type in match_types:  
        dict_string = f"{match_type}"
        match_type_data = player[4][dict_string]
        
        if match_type_data == None:    # for singes and doubles stats, if no activity is recorded, put None for stats variables
            for stats_list in valid_sets[dict_string]:
                stats_list.append(None)
        else:
            for i in range(5): 
                valid_sets[dict_string][i].append(match_type_data[i])              
            
'''
    


print(f"Players left for test set: {len(name_list)}")


[15.3, 47.0, -4.11, 61.04, 5]
[14.36, -12.0, -21.34, 46.15, 3]
[14.99, 53.0, 19.53, 61.43, 4]
[14.47, -10.0, -15.47, 25.0, 1]
[15.27, -48.0, -62.49, 42.86, 5]
[14.33, 9.0, -6.96, 42.86, 2]
[4.21, -15.0, -10.5, 28.57, 2]
[4.34, 36.0, 26.55, 50.72, 5]
[4.75, -33.0, -34.9, 0.0, 0]
[6.03, -209.0, -262.0, 0.0, 0]
[15.19, -22.0, -31.82, 48.33, 5]
[14.3, -6.0, -10.74, 36.36, 3]
[15.51, 431.0, 305.03, 92.5, 29]
[14.47, 18.0, 1.84, 55.56, 2]
[8.14, 28.0, 21.6, 63.64, 4]
None
[3.17, 27.0, 20.1, 60.0, 4]
[3.0, -3.0, -4.7, 50.0, 1]
[7.5, 5.0, 6.0, 50.0, 1]
[7.0, 5.0, 4.45, 100.0, 1]
[4.9, -4.0, -1.4, 40.0, 2]
None
[6.53, 52.0, 22.5, 62.07, 5]
[4.5, -10.0, -11.0, 0.0, 0]
[15.33, 169.0, 128.78, 72.31, 7]
[14.45, -6.0, -7.66, 25.0, 1]
[15.34, 146.0, 99.42, 67.11, 9]
[14.04, 5.0, -10.7, 47.06, 3]
[15.01, -11.0, 27.93, 44.19, 4]
[13.9, -23.0, -27.34, 20.0, 1]
[14.68, 77.0, -79.3, 66.22, 10]
[14.06, -6.0, -37.23, 36.84, 3]
[14.96, -21.0, -10.53, 48.61, 4]
[14.13, -1.0, -8.03, 36.36, 2]
[7.36, 32.0, 31.6

In [9]:
print(f"elements in data: {len(data)}")
print(f"Elements left to use for test set: {len(name_list)}")

elements in data: 76
Elements left to use for test set: 6


In [10]:
scaler = MinMaxScaler(feature_range=(0,1))


def scaler_func(list_, scaled_list):

    # list_ = np.array(list_)
    '''
    for elem in list_:
        if elem != 'No data?' and elem != None:
            elem = float(elem)
        else:
            elem = np.array([0])
    '''
    
    # list_ = [float(x) if x != 'No data?' and x != None else np.array([0]) for x in list_]

    # print(list_)
    numeric_data = [x if isinstance(x, (int, float)) else 0 for x in list_]  # creates a new list numeric_data that contains all values that are intagers and floats
    numeric_data = np.array(numeric_data).reshape(-1, 1)  # Converts all numbers in training set to numpy. Training sets with None/string values can't be converted to numpy (I think)
    scaled_numeric_data = scaler.fit_transform((numeric_data))
    
    # scaled_list = scaled_basic_training_sets[i]
    numeric_index = 0

    '''
    for item in list_:     # If item in basic_training_sets list is string or None, append that. If item is int or float, append next scaled_numeric_data value 
        if isinstance(item, (int, float)):
            scaled_list.append(scaled_numeric_data[numeric_index])
            numeric_index += 1
        else:
            scaled_list.append(np.array([0]))
    '''

    # print(f"scaled_list = {scaled_list}")

    # scaled_list.clear()

    # print(scaled_list)
    '''
    if scaled_list != None:
        scaled_list[:] = scaled_numeric_data.reshape(70,1) # manually changed the shape
    else:
        scaled_list = scaled_numeric_data.flatten()
    '''

    # scaled_list = [0 if x != 'No data?' or x != None else x for x in scaled_list]
    
    # print(f"scaled_numeric_data = {scaled_numeric_data}")

    return scaled_numeric_data
            



# serveSpeed_train_labels = np.array(serveSpeed_train_labels)

print(f"pre-scaled serve speed training labels: {type(scaled_serveSpeed_train_labels)}")

scaled_serveSpeed_train_labels = scaler_func(serveSpeed_train_labels, scaled_serveSpeed_train_labels)

print(f"Scaled serve speed training lables: {type(scaled_serveSpeed_train_labels)}")


# scaled_serveSpeed_train_labels = scaler.fit_transform((serveSpeed_train_labels).reshape(-1,1))


for i in range(3):   # Adding scaled values to all basic training sets
    list_ = basic_training_sets[i]
    scaled_list = scaled_basic_training_sets[i]

    print(f"pre-scaled scaled_basic_training_sets[0] = {type(scaled_basic_training_sets[0])}")

    scaled_basic_training_sets[i] = scaler_func(basic_training_sets[i], scaled_basic_training_sets[i])

    # scaled_basic_training_sets[i] = np.array(scaled_basic_training_sets[i]).reshape(-1, 1)

    print(f"basic_training_set scaled list type: {type(scaled_basic_training_sets[i])}")
    print(f"basic_training_set scaled list: {scaled_basic_training_sets[i]}")


print(f"scaled_singlesUtr_train_samples type = {type(scaled_singlesUtr_train_samples)}")
print(f"scaled_singlesUtr_train_samples = {scaled_singlesUtr_train_samples}")


for match_type in match_types:  
    dict_string = f"{match_type}"
    dict_string = 'singles'    # CHANGE LATER 
    # match_type_data = player[4][dict_string]
    '''
    if adv_training_sets[dict_string][0] == None:    # for singes and doubles stats, if no activity is recorded, skip
        print(f"Empty set: {adv_training_sets[dict_string][0]}")
    else:
    '''
    for i in range(5):   # Adding scaled values to all advanced stat training sets
        list_ = adv_training_sets[match_type][i]
        scaled_list = scaled_adv_training_sets[dict_string][i]
        scaled_adv_training_sets[dict_string][i].extend(scaler_func(list_, scaled_list))
        # scaled_adv_training_sets[dict_string][i] = scaler.fit_transform((scaled_adv_training_sets[dict_string][i]).reshape(-1,1))
        print(f"adv_training_set scaled list: {type(scaled_adv_training_sets[dict_string][i])}")

      



pre-scaled serve speed training labels: <class 'list'>
Scaled serve speed training lables: <class 'numpy.ndarray'>
pre-scaled scaled_basic_training_sets[0] = <class 'list'>
basic_training_set scaled list type: <class 'numpy.ndarray'>
basic_training_set scaled list: [[0.60759494]
 [0.46582278]
 [0.55949367]
 [0.06329114]
 [0.86075949]
 [0.67594937]
 [0.5164557 ]
 [0.7721519 ]
 [0.65822785]
 [0.75949367]
 [0.58227848]
 [1.        ]
 [0.50379747]
 [0.61772152]
 [0.73670886]
 [0.60506329]
 [0.46835443]
 [0.27848101]
 [0.47848101]
 [0.50379747]
 [0.70379747]
 [0.63544304]
 [0.61265823]
 [0.62025316]
 [0.49113924]
 [0.56708861]
 [0.45822785]
 [0.44556962]
 [0.5443038 ]
 [0.49113924]
 [0.52658228]
 [0.40506329]
 [0.58481013]
 [0.63291139]
 [0.25316456]
 [0.4835443 ]
 [0.49873418]
 [0.67594937]
 [0.54936709]
 [0.49113924]
 [0.55696203]
 [0.48101266]
 [0.94936709]
 [0.43037975]
 [0.45316456]
 [0.52911392]
 [0.20253165]
 [0.57468354]
 [0.        ]
 [0.46582278]
 [0.57721519]
 [0.59493671]
 [0.29

In [11]:
print(scaled_serveSpeed_train_labels)

[[0.8378633 ]
 [0.        ]
 [0.90990795]
 [0.40950931]
 [0.3805175 ]
 [0.90541422]
 [0.87837936]
 [0.54359643]
 [0.43125317]
 [0.44937305]
 [0.4421251 ]
 [0.43850112]
 [0.90990795]
 [0.86040444]
 [0.        ]
 [0.        ]
 [0.86489817]
 [0.54359643]
 [0.84235703]
 [0.84235703]
 [1.        ]
 [0.86040444]
 [0.        ]
 [0.47474089]
 [0.86040444]
 [0.91896789]
 [0.86489817]
 [0.        ]
 [0.        ]
 [0.87837936]
 [0.8874393 ]
 [0.51098065]
 [0.86040444]
 [0.456621  ]
 [0.36964558]
 [0.8693919 ]
 [0.87837936]
 [0.82887584]
 [0.        ]
 [0.90092049]
 [0.4421251 ]
 [0.49648474]
 [0.46386896]
 [0.89642676]
 [0.8874393 ]
 [0.88287309]
 [0.44937305]
 [0.83336957]
 [0.43125317]
 [0.91447416]
 [0.88287309]
 [0.33340581]
 [0.49648474]
 [0.86489817]
 [0.31166196]
 [0.91896789]
 [0.5182286 ]
 [0.88287309]
 [0.8378633 ]
 [0.87837936]
 [0.        ]
 [0.87388563]
 [0.89193303]
 [0.50735667]
 [0.86489817]
 [0.94600275]
 [0.84235703]
 [0.8693919 ]
 [0.90092049]
 [0.89193303]]


In [12]:
'''

print(scaled_serveSpeed_train_labels.shape)
print(scaled_singlesUtr_train_samples.shape)
print(scaled_doublesUtr_train_samples.shape)
print(scaled_servePercentage_train_samples.shape)
print(scaled_singlesGameDiff_train_samples.shape)
print(scaled_singlesWeightedGameDiff_train_samples.shape)
print(scaled_singlesWinPercentage_train_samples.shape)
print(scaled_singlesOpponentUtr_train_samples.shape)
print(scaled_singlesLongestWinStreak_train_samples.shape)



print(len(scaled_singlesUtr_train_samples))
print(len(scaled_doublesUtr_train_samples))
print(len(scaled_servePercentage_train_samples))
print(len(scaled_singlesGameDiff_train_samples))
print(len(scaled_singlesWeightedGameDiff_train_samples))
print(len(scaled_singlesWinPercentage_train_samples))
print(len(scaled_singlesOpponentUtr_train_samples))
print(len(scaled_singlesLongestWinStreak_train_samples))

'''

print(type(scaled_singlesUtr_train_samples))
print(type(scaled_doublesUtr_train_samples))
print(type(scaled_servePercentage_train_samples))
print(type(scaled_singlesGameDiff_train_samples))
print(type(scaled_singlesWeightedGameDiff_train_samples))
print(type(scaled_singlesWinPercentage_train_samples))
print(type(scaled_singlesOpponentUtr_train_samples))
print(type(scaled_singlesLongestWinStreak_train_samples))
print(type(scaled_serveSpeed_train_labels))


'''

print(scaled_singlesUtr_train_samples)
print(scaled_doublesUtr_train_samples)
print(scaled_servePercentage_train_samples)
print(scaled_singlesGameDiff_train_samples)
print(scaled_singlesWeightedGameDiff_train_samples)
print(scaled_singlesWinPercentage_train_samples)
print(scaled_singlesOpponentUtr_train_samples)
print(scaled_singlesLongestWinStreak_train_samples)
print(scaled_serveSpeed_train_labels)
'''

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'numpy.ndarray'>


'\n\nprint(scaled_singlesUtr_train_samples)\nprint(scaled_doublesUtr_train_samples)\nprint(scaled_servePercentage_train_samples)\nprint(scaled_singlesGameDiff_train_samples)\nprint(scaled_singlesWeightedGameDiff_train_samples)\nprint(scaled_singlesWinPercentage_train_samples)\nprint(scaled_singlesOpponentUtr_train_samples)\nprint(scaled_singlesLongestWinStreak_train_samples)\nprint(scaled_serveSpeed_train_labels)\n'

### Making the Model

In [13]:
end_sets = np.column_stack((scaled_singlesUtr_train_samples, scaled_doublesUtr_train_samples, scaled_servePercentage_train_samples,
                           scaled_singlesGameDiff_train_samples, scaled_singlesWeightedGameDiff_train_samples, 
                            scaled_singlesWinPercentage_train_samples, scaled_singlesOpponentUtr_train_samples, 
                            scaled_singlesLongestWinStreak_train_samples))

print(end_sets)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 0 and the array at index 3 has size 140

In [ ]:
'''
model = Sequential([
    Input(shape=(1,)),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
'''

In [ ]:
# model.compile(Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(scaled_train_samples, train_labels, validation_split=0.1, batch_size=10, epochs=20, shuffle=True, verbose=2)